In [2]:
%matplotlib inline

from pathlib import Path 

import dask.dataframe as dd
import pandas as pd

In [3]:
YEAR = 2019

In [4]:
slookup = pd.read_csv('ghcn_mos_lookup.csv')

# GHCN

In [5]:
names = ['ID', 'DATE', 'ELEMENT', 'DATA_VALUE', 'M-FLAG', 'Q-FLAG', 'S-FLAG', 'OBS-TIME']
ds = dd.read_csv(f's3://noaa-ghcn-pds/csv/{YEAR}.csv', storage_options={'anon':True},  
                 names=names, parse_dates=['DATE'], dtype={'DATA_VALUE':'object'})

ghcn = ds[['ID', 'DATE', 'ELEMENT', 'DATA_VALUE']][ds['ID'].isin(slookup['ID']) & ds['ELEMENT'].str.match('TAVG')].compute()

In [6]:
ghcn.head()

,ID,DATE,ELEMENT,DATA_VALUE
61957,USC00244558,2019-01-01,TAVG,-74
85474,USW00003017,2019-01-01,TAVG,-155
85728,USW00003170,2019-01-01,TAVG,-144
85862,USW00003810,2019-01-01,TAVG,154
85885,USW00003812,2019-01-01,TAVG,145


# MOS

In [7]:
file_list = list(Path(f'station_filter').glob("*.csv"))

In [8]:
columns = ['station', 'short_model', 'model', 'runtime', 'ftime', 'N/X', 'X/N',
           'TMP', 'DPT', 'WDR', 'WSP', 'CIG', 'VIS', 'P06', 'P12', 'POS', 'POZ',
           'SNW', 'CLD', 'OBV', 'TYP', 'Q06', 'Q12', 'T06', 'T12']
usecols = ['station', 'runtime', 'ftime', 'TMP']

In [9]:
df = pd.read_csv(file_list[0], names=columns, usecols=usecols).drop_duplicates().dropna()

C:\Users\story\Anaconda3\envs\mpp\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [10]:
mask = df['runtime'].str.contains('2019') & df['ftime'].str.contains('2019')
dt = pd.to_datetime(df['ftime'][mask])
dt2 = pd.to_datetime(df['runtime'][mask])

In [11]:
((dt.dt.tz_localize('UTC') - dt2).dt.total_seconds()/3600).astype(int)

0           6
1           9
2          12
3          15
4          18
           ..
3682545    54
3682546    57
3682547    60
3682548    66
3682549    72
Length: 2275646, dtype: int32

In [12]:
mos_tables = []
for f in sorted(file_list):
    mos = pd.read_csv(f, names=columns, usecols=usecols).drop_duplicates().dropna()
    #somehow there's a row where the header names got repeated
    mos.drop(mos[mos['station'].str.match('station')].index, inplace=True)
    mask = mos['runtime'].str.contains('2019') & mos['ftime'].str.contains('2019')
    mos.drop(mos[~mask].index, inplace=True)
    mos
        
    #filter & convert
    mosc = mos[['station', 'TMP']].astype({'TMP':float})
    mosc['runtime'] = pd.to_datetime(mos['runtime'])
    mosc['ftime'] = pd.to_datetime(mos['ftime'])
    mosc['hours'] = ((mosc['ftime'].dt.tz_localize('UTC') - mosc['runtime']).dt.total_seconds()/3600).astype(int)
    mosc['date'] = mosc['ftime'].dt.date
   
    mosc['TMP'] = mosc['TMP']
    mos_tables.append(mosc)

C:\Users\story\Anaconda3\envs\mpp\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [14]:
mos_all = pd.concat(mos_tables)

In [154]:
mos_all.to_csv("filtered_mos.csv", index=False)

In [15]:
mos_all.head()

,station,TMP,runtime,ftime,hours,date
0,NSTU,83.0,2019-01-01 00:00:00+00:00,2019-01-01 06:00:00,6,2019-01-01
1,NSTU,84.0,2019-01-01 00:00:00+00:00,2019-01-01 09:00:00,9,2019-01-01
2,NSTU,83.0,2019-01-01 00:00:00+00:00,2019-01-01 12:00:00,12,2019-01-01
3,NSTU,83.0,2019-01-01 00:00:00+00:00,2019-01-01 15:00:00,15,2019-01-01
4,NSTU,84.0,2019-01-01 00:00:00+00:00,2019-01-01 18:00:00,18,2019-01-01


In [16]:
slook = slookup.rename(columns={'Station':'station'})[['ID', 'station']]
slook.dtypes

ID         object
station    object
dtype: object

In [17]:
mos_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62283443 entries, 0 to 1269436
Data columns (total 6 columns):
 #   Column   Dtype              
---  ------   -----              
 0   station  object             
 1   TMP      float64            
 2   runtime  datetime64[ns, UTC]
 3   ftime    datetime64[ns]     
 4   hours    int32              
 5   date     object             
dtypes: datetime64[ns, UTC](1), datetime64[ns](1), float64(1), int32(1), object(2)
memory usage: 3.0+ GB


In [18]:
moswn = pd.merge(mos_all, slook, how='right', on='station')

In [19]:
moswn['datestr'] = moswn['date'].astype(str)

In [20]:
mtable = moswn[[ 'ID', 'datestr', 'station','hours', 'TMP']].pivot_table(index=["ID", 'datestr', 'station'], columns=['hours'], values='TMP')

In [21]:
ghcn['DATESTR'] = ghcn['DATE'].astype(str)

In [22]:
ghcn.head()

,ID,DATE,ELEMENT,DATA_VALUE,DATESTR
61957,USC00244558,2019-01-01,TAVG,-74,2019-01-01
85474,USW00003017,2019-01-01,TAVG,-155,2019-01-01
85728,USW00003170,2019-01-01,TAVG,-144,2019-01-01
85862,USW00003810,2019-01-01,TAVG,154,2019-01-01
85885,USW00003812,2019-01-01,TAVG,145,2019-01-01


In [24]:
pairup = pd.merge(mtable.reset_index(), ghcn, how='inner', right_on=['ID', 'DATESTR'], left_on=['ID', 'datestr'])

In [25]:
pairup.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 102450 entries, 0 to 102449
Data columns (total 28 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   ID          102450 non-null  object        
 1   datestr     102450 non-null  object        
 2   station     102450 non-null  object        
 3   6.0         102450 non-null  float64       
 4   9.0         102450 non-null  float64       
 5   12.0        102450 non-null  float64       
 6   15.0        102450 non-null  float64       
 7   18.0        102450 non-null  float64       
 8   21.0        102450 non-null  float64       
 9   24.0        102173 non-null  float64       
 10  27.0        102173 non-null  float64       
 11  30.0        102173 non-null  float64       
 12  33.0        102173 non-null  float64       
 13  36.0        102173 non-null  float64       
 14  39.0        102173 non-null  float64       
 15  42.0        102173 non-null  float64       
 16  45

In [26]:
pairup.columns

Index([        'ID',    'datestr',    'station',          6.0,          9.0,
               12.0,         15.0,         18.0,         21.0,         24.0,
               27.0,         30.0,         33.0,         36.0,         39.0,
               42.0,         45.0,         48.0,         51.0,         54.0,
               57.0,         60.0,         66.0,         72.0,       'DATE',
          'ELEMENT', 'DATA_VALUE',    'DATESTR'],
      dtype='object')

In [39]:
hours = [6.0, 9.0, 12.0, 15.0, 18.0, 21.0, 24.0, 27.0, 30.0, 33.0, 36.0, 39.0, 42.0, 45.0, 48.0, 51.0, 54.0, 57.0, 60.0, 66.0, 72.0]
hc = {i:str(int(i)) for i in hours}

In [41]:
columns = {'datestr':'date', 'DATA_VALUE':'observed'}
columns.update(hc)
table = pairup[['station', 'datestr', 'DATA_VALUE']+hours].rename(columns=columns)

In [42]:
table

,station,date,observed,6,9,12,15,18,21,24,...,39,42,45,48,51,54,57,60,66,72
0,KGPI,2019-01-01,-74,17.333333,18.00,15.50,17.50,19.00,24.00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,KGPI,2019-01-02,-46,23.000000,23.20,23.20,23.20,23.20,24.60,22.00,...,23.50,25.00,31.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,KGPI,2019-01-03,-3,28.200000,29.60,29.40,29.20,30.60,31.20,30.60,...,29.20,30.20,32.40,30.00,29.75,27.333333,29.333333,28.5,33.00,NaN
3,KGPI,2019-01-04,38,38.800000,38.00,37.00,37.00,37.40,38.20,35.80,...,33.60,34.40,35.80,35.20,34.00,33.600000,33.200000,32.8,33.60,34.50
4,KGPI,2019-01-05,3,34.200000,32.40,33.40,33.00,34.60,35.80,36.20,...,30.80,31.80,32.80,33.60,32.60,32.000000,31.400000,32.2,32.40,33.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102445,KALO,2019-12-27,8,30.000000,29.50,29.50,29.25,28.75,28.75,28.75,...,29.25,29.75,29.75,30.75,30.50,31.000000,32.000000,33.0,32.75,32.75
102446,KALO,2019-12-28,13,33.000000,33.00,32.50,33.50,33.00,33.50,32.50,...,34.00,33.75,34.50,34.75,35.75,35.250000,37.000000,37.5,38.25,37.00
102447,KALO,2019-12-29,76,46.250000,46.00,45.50,47.00,47.25,47.50,47.75,...,48.00,47.25,48.00,47.00,48.75,49.000000,49.000000,48.5,49.25,47.50
102448,KALO,2019-12-30,-11,30.500000,29.25,31.25,29.00,30.50,29.00,30.75,...,30.25,33.00,31.25,34.00,31.25,32.250000,30.750000,34.0,33.50,32.75


In [44]:
table.to_csv("ALL_2019.csv", index=False)